In [2]:
# RUN ON 'DCL' venv
import pandas as pd

In [3]:
df = pd.read_csv("../Davidson-Original/data/labeled_data.csv")

In [3]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [4]:
# Class distribution
df['class'].value_counts()

# `class` = class label for majority of CF users.
#   0 - hate speech
#   1 - offensive  language
#   2 - neither


class
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [5]:
# As percentages
df['class'].value_counts(normalize=True) * 100

class
1    77.432111
2    16.797805
0     5.770084
Name: proportion, dtype: float64

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BartForConditionalGeneration, BartTokenizer

# Initialize Language Model and Tokenizer
lm_model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

c:\Users\Owner\anaconda3\envs\DCL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 3.65kB/s]


In [10]:
# Initialize Summarization Model and Tokenizer
summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarization_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

model.safetensors:  14%|█▍        | 231M/1.63G [06:49<41:54, 555kB/s] Error while downloading from https://cdn-lfs.huggingface.co/facebook/bart-large-cnn/40041830399afb5348525ef8354b007ecec4286fdf3524f7e6b54377e17096cb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1711720149&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTcyMDE0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9mYWNlYm9vay9iYXJ0LWxhcmdlLWNubi80MDA0MTgzMDM5OWFmYjUzNDg1MjVlZjgzNTRiMDA3ZWNlYzQyODZmZGYzNTI0ZjdlNmI1NDM3N2UxNzA5NmNiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=yrNo0H-kgCWjuj3Eq0paxIe%7EMdIWZBvO8LZMC5dwKa7XHHKztI1Ev2ScvstNxsaIbW-xNakJ7XqIQb-TWqTHwYcr8K0efZwYtQTOmjkSaKmjg%7EiesZxhyAbzzEthtNY%7Ev9%7EOvMGoe2kzmXhmdzPeX5VvW%7EF6kQ%7E9iEy09ps3y1rvjmXbu--8Thdet42j64Dg8wGBwNhK0beL1Xeg7ILShfZQltOQyuOpvLMdbSfZ8LDK9D-bEo%7EFs287UZ7yxsstvIhqv7NkFtop1KB-%7EwIRZKU7fcb

KeyboardInterrupt: 

In [ ]:
# Function to generate long phrases from hate speech
def generate_long_phrases(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = lm_model.generate(inputs, max_length=200, num_return_sequences=1, early_stopping=True)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Function to generate tweet representations from long phrases
def generate_tweet_representations(long_phrases):
    tweet_representations = []
    for phrase in long_phrases:
        input_ids = summarization_tokenizer.encode("summarize: " + phrase, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = summarization_model.generate(input_ids, max_length=50, min_length=10, num_beams=4, early_stopping=True)
        tweet_representations.append(summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True))
    return tweet_representations

In [ ]:
# Generate synthetic hate speech samples
synthetic_samples = []
hate_speech_samples = df[df['class'] == 0]['tweet'].tolist()
hate_speech_samples = hate_speech_samples[:5] # TESTING
for sample in hate_speech_samples:
    print("Original Tweet:", sample)
    long_phrases = generate_long_phrases(sample)
    print("Long Phrases:", long_phrases)
    tweet_representations = generate_tweet_representations(long_phrases)
    print("Tweet Representations:", tweet_representations)
    synthetic_samples.extend(tweet_representations)

In [ ]:
# # Add synthetic samples to dataset
# synthetic_df = pd.DataFrame({'text': synthetic_samples, 'class': [0] * len(synthetic_samples)})
# df_augmented = pd.concat([df, synthetic_df], ignore_index=True)

# # Check class distribution after augmentation
# print(df_augmented['class'].value_counts())